# RAG implementado con Langchain, base de datos vectorial Chroma, embeddings de HuggingFace y API de Gemini.

* Se usa el dataset de transcripciones de podcast de Spotify.
* Se cuenta también con un archivo de validación, que consiste en preguntas y las respuestas "correctas". Se usará estas respuestas para chequear que el RAG funciona bien.

Referencias:
* https://docs.mistral.ai/guides/rag/#rag-from-scratch
* https://python.langchain.com/docs/integrations/vectorstores/chroma/
* https://medium.com/@callumjmac/implementing-rag-in-langchain-with-chroma-a-step-by-step-guide-16fc21815339

In [2]:
import numpy as np
import pandas as pd
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document # Importing Document schema from Langchain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import google.generativeai as genai
import os

/home/lucas/miniconda3/envs/NLP/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Veo el archivo de validación:

In [2]:
df_qa = pd.read_csv('acquired-qa-evaluation.csv')
df_qa.head()

,question,human_answer,ai_answer_without_the_transcript,ai_answer_without_transcript_correctness,ai_answer_with_the_transcript,ai_answer_with_the_transcript_correctness,quality_rating_for_answer_with_transcript,post_url,file_name,Unnamed: 9
0,"When did Airbnb go public, what was the price ...","December 9,2020 at $68 per share","Airbnb went public on December 10, 2020. The i...",CORRECT,"Airbnb went public in 2020. However, the speci...",INCORRECT,4,https://www.acquired.fm/episodes/airbnb,airbnb,NaN
1,Why did Wimdu unlike Airbnb not take off?,Wimdu gragmented the marketed focusing mostly ...,Wimdu faced challenges compared to Airbnb due ...,CORRECT,"Wimdu, similar to Airbnb, was a platform creat...",CORRECT,5,https://www.acquired.fm/episodes/airbnb,airbnb,NaN
2,Why does market fragmentation work for airline...,Even though both the airline industry and airb...,Market fragmentation benefits the airline indu...,CORRECT,Market fragmentation can work for the airline ...,CORRECT,3,https://www.acquired.fm/episodes/airbnb,airbnb,NaN
3,How many hot dogs does Costco currently sell p...,130 million,Costco sold just shy of 200 million hot dog an...,INCORRECT,Annual Hot Dog Sales: Costco sells 130 million...,CORRECT,5,https://www.acquired.fm/episodes/costco,costco,NaN
4,"What store was created as ""the price club of h...",Home Depot,"The store created as the ""price club of hardwa...",CORRECT,"Store Created as ""the price club of hardware s...",CORRECT,5,https://www.acquired.fm/episodes/costco,costco,NaN


In [3]:
df_qa.shape

(80, 10)

In [4]:
df_qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 10 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   question                                   80 non-null     object
 1   human_answer                               80 non-null     object
 2   ai_answer_without_the_transcript           80 non-null     object
 3   ai_answer_without_transcript_correctness   80 non-null     object
 4   ai_answer_with_the_transcript              80 non-null     object
 5   ai_answer_with_the_transcript_correctness  80 non-null     object
 6   quality_rating_for_answer_with_transcript  80 non-null     object
 7   post_url                                   80 non-null     object
 8   file_name                                  80 non-null     object
 9   Unnamed: 9                                 1 non-null      object
dtypes: object(10)
memory usage: 6.4+ KB


In [5]:
df_qa.file_name.value_counts()

file_name
qualcomm                                                6
enron                                                   5
spacex                                                  5
bitcoin                                                 4
airbnb                                                  3
costco                                                  3
disney_plus                                             3
whatsapp                                                3
berkshire_hathaway_part_i                               3
nvidia_part_iii_the_dawn_of_the_ai_era_20222023         3
nvidia_part_ii_the_machine_learning_company_20062022    3
ethereum_with_packy_mccormick                           3
nvidia_part_i_the_gpu_company_19932006                  3
walmart                                                 3
renaissance_technologies                                3
nintendos_origins                                       3
visa                                                    3
amaz

Son muchos archivos, así que me quedo solamente con los primeros 5 file_names:

In [6]:
file_names = ['qualcomm','enron','spacex','bitcoin','airbnb']
df_qa.file_name.isin(file_names).sum()

23

Creo un nuevo dataframe df_qf donde me quedo solamente con los datos correspondiente a estos 5 file_names:

In [7]:
df_qf = df_qa[['question','file_name','human_answer']][df_qa.file_name.isin(file_names)]
df_qf.head()

,question,file_name,human_answer
0,"When did Airbnb go public, what was the price ...",airbnb,"December 9,2020 at $68 per share"
1,Why did Wimdu unlike Airbnb not take off?,airbnb,Wimdu gragmented the marketed focusing mostly ...
2,Why does market fragmentation work for airline...,airbnb,Even though both the airline industry and airb...
9,"According to Information Theory, what is the i...",qualcomm,The more closely the actual communication is t...
10,Compare the impact on Qualcomm between the two...,qualcomm,Erwin Jacobs was a genius and visionary who pa...


In [54]:
len(file_names)

5

Tengo 5 archivos con 29 preguntas, con sus respectivas respuestas "correctas": respuesta humana 'human_answer' y los nombres de los archivos donde se encuentran las respuestas 'file_name'. Con estos dos datos voy a testear mi RAG. A partir de las preguntas testeo si el RAG: 
* busca la respuesta en el archivo correcto.
* la distancia coseno entre la respuesta humana y la del LLM.

## Ahora leo las transcripciones individuales y las spliteo

In [9]:
PATH = "../../acquired-individual-transcripts/"

docs = []

for file_name in file_names:
    with open(PATH+file_name+'.txt','r') as file:
        text = file.read()
        doc =  Document(page_content=text[113:], metadata={"source": file_name})
        docs.append(doc)

len(docs)

5

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
print(f"Split {len(docs)} documents into {len(chunks)} chunks.")

Split 5 documents into 886 chunks.


In [11]:
chunks[0]

Document(metadata={'source': 'qualcomm'}, page_content="David: I walked in. The first thing I saw was the bottom of the big crane boom arm with the weights. I was like, why are there Olympic weights here? And then I was like, oh, because we got a professional boom arm camera. This is amazing. Ben: All right, let's do it. Welcome to season 11, episode 6 of Acquired, the podcast about great technology companies and the stories and playbooks behind them. I'm Ben Gilbert. I'm the Co-Founder and Managing Director of Seattle-based Pioneer Square Labs and our venture fund, PSL Ventures. David: I'm David Rosenthal. I'm an angel investor based in San Francisco. Ben: And we are your hosts. There's an incredible property of the universe where electromagnetic signals can be broadcast and travel through space at the speed of light to be received at a different point in the universe. Now, a tiny fraction of these frequencies are detectable by humans as visible light.  Some other frequencies can be d

## Creo la base de datos con los chunks y embeddings

In [83]:
# cargo los embeddings en local. modelo: multi-qa-MiniLM-L6-cos-v1
model_name = "/home/lucas/Documentos/Lucas/ML/embeddings/Sentence Transformers/"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

In [14]:
# creo la base de datos Chroma con los documentos y embeddings
#db = Chroma.from_documents(collection_name='chroma_lc_rag',documents=chunks, embedding=embeddings, persist_directory='chroma_db')

In [34]:
# ahora puedo cargar la base de datos
db = Chroma(collection_name='chroma_lc_rag',persist_directory="chroma_db", embedding_function=embeddings)

In [35]:
# pruebo que la base de datos funciona bien
question = 'what is the idea about mars oasis'
sim = db.similarity_search(question, k=1)
sim[0].page_content

"this is their charter, so he makes a $100,000 donation to The Mars Society. He joins the board and he starts meeting all of these aerospace people in LA. Not just in LA, of course back up in Silicon Valley there's NASA's Jet Propulsion Lab in Mountain View. \xa0 Elon's mostly down in LA but he's going back and forth. He starts organizing these “Saturday salons,” he calls them, where he's just getting together industry leaders in aerospace and at JPL, both in LA and Palo Alto. There's no agenda, but he lets it be known to all of them that he's got some resources. He's a dot-com-rich guy and he wants to make a gesture. What could be done on the order of $10–$20 million.  They start to coalesce the group on this idea of building a “Mars Oasis” and the idea behind a Mars Oasis is that they're going to buy a rocket, and they're going to put a plant on it, and they're also going to put a robot on it, and they're going to shoot this rocket to mars. I can't remember if the Mars Rover had land

#### Métricas de la base de datos

Chequeo al hacer una pregunta, el chunk que devuelve Chroma esté en el documento correcto.

In [36]:
response_file_name = []
for question in df_qf.question:
    sim = db.similarity_search(question, k=1)
    response_file_name.append(sim[0].metadata['source'])

response_file_name = pd.Series(response_file_name)
print(response_file_name.shape)
print(df_qf.file_name.shape)

(23,)
(23,)


In [52]:
from sklearn.metrics import accuracy_score
print('accuracy = ',accuracy_score(response_file_name,df_qf.file_name))

accuracy =  1.0


Osea que el 100% de los chunks devueltos por la base de datos, pertenecen al documento correcto.

## Función para consultar:

In [4]:
# configuro la api de gemini
# console: export GOOGLE_API_KEY="***api_key***"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel('gemini-1.5-pro')
model.generate_content('hello! who are you?').text

"I am a large language model, trained by Google.  I don't have a name or a personal identity. I'm an AI that processes information and responds to a wide range of prompts and questions.  I can provide summaries of factual topics, create stories, and translate languages.  I'm still under development, but I'm learning new things every day.\n"

In [81]:
# creo función para hacer consultas a la api de gemini
def consulta(question):
    sim = db.similarity_search(question, k=1)
    retrieved_chunk = sim[0]
    
    prompt = f"""
    Given the context information and not prior knowledge, answer the query.
    Context information:
    ---------------------
    {retrieved_chunk.page_content}
    ---------------------
    Query: {question}
    Answer:
    """
    response = model.generate_content(prompt)
    
    return retrieved_chunk, response.text

Y la uso para hacer una nueva pregunta:

In [82]:
question_number=9
question = df_qf['question'].iloc[question_number]
print('question: ',question)
print('------')
chunk, response = consulta(question)
print('response: ',response)
print('chunk_source: ',chunk.metadata['source'])

question:  what is the idea about mars oasis
------
response:  The idea behind Mars Oasis is to buy a rocket, put a plant and a robot on it, and send it to Mars.  The goal is to create a miniature oasis on the planet as a demonstrative gesture.

chunk_source:  spacex


## Conclusiones:

* Se cargaron los archivos a documentos de LangChain y se splitearon en 'chunks'.
* Se creó la base de datos vectorial Chroma, donde se encuentra los chunks y los embeddings. El modelo de embeddings "multi-qa-MiniLM-L6-cos-v1" se obtuvo de HuggingFace.
* Se testeó la performance de la base vectorial, obteniendo que apunta al archivo correctamente en el 100% de los casos.
* Se implementó el RAG usando la base de datos vectorial Chroma y la API de Gemini.
* Se probó el RAG con varias de las preguntas de validación y pareciera funcionar bien en general.

Trabajo a futuro:
* Faltaría hacer métricas de la performance del RAG, chequeando que las respuestas obtenidas sean las esperadas, usando las respuestas "humanas" del archivo de validación.
* La selección de la longitud de los 'chunks' (chunk_size) es una variable importante que se puede ajustar en función de la performance obtenida de la base de datos vectorial.